<a href="https://colab.research.google.com/github/tmsk0711/kaggle_prec/blob/main/%08Strategy_prec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import warnings
warnings.simplefilter('ignore')
import tensorflow as tf

import numpy as np
import os

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


# 하나의 차원을 배열에 추가 -> 새로운 shape == (28, 28, 1)
# 이렇게 하는 이유는 우리의 모델에서 첫 번째 층이 합성곱 층이고
# 합성곱 층은 4D 입력을 요구하기 때문입니다.
# (batch_size, height, width, channels).
# batch_size 차원은 나중에 추가할 것입니다.

train_images = train_images[..., None]
test_images = test_images[..., None]

# 이미지를 [0, 1] 범위로 변경하기.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

4431872/4422102 [==============================] - 0s 0us/step


# strategy.scope() ??
https://www.tensorflow.org/api_docs/python/tf/distribute/Strategy

> * 분산전략? 분산과관련된 처리를 한다?
> * 모든 변수와 모델, 그래프는 strategy(지정한 변수)에 복제된다.
> * strategy는 주어지는 입력에 대해서 손실(loss)와 gradient(가중치)를 계산한다.
> * gradient(가중치)를 전부 더함으로써 모든 장치(GPU,TPU)간에 gradient(가중치)가 동기화 된다.
> * 동기화된 후에, 동일한 업데이트가 각 장치에 있는 변수의 복사본(copies)에 동일하게 적용

# 결론
> * 훈련을 여러 TPU,GPU로 나눠 처리하고 각자 장치간의 훈련손실, 가중치를 기록하고 계산하기 위함
> * 나눠 처리된 훈련을 테스트 손실과 훈련 정확도,체크포인트,등 이해하고 처리 할 수 있다.
> * MirroedStrategy를 사용하면 미러링된 변수들을 하나의 가상변수에 대응되어서, 모두 같은 값을 유지 된다고 한다.

In [ ]:
# 만약 장치들의 목록이 `tf.distribute.MirroredStrategy` 생성자 안에 명시되어 있지 않다면,
# 자동으로 장치를 인식할 것입니다.
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
print ('장치의 수: {}'.format(strategy.num_replicas_in_sync))


장치의 수: 1


# 입력 파이프 라인 설정
> 그래프와 변수를 플랫폼과 무관한 SavedModel 형식으로 내보냅니다. 모델을 내보냈다면, 모델을 불러올 때 범위(scope)를 지정하지 않아도 된다.

In [ ]:
BUFFER_SIZE = len(train_images)
BUFFER_SIZE # 60000

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
GLOBAL_BATCH_SIZE # 64

EPOCHS = 10

>분산 데이터셋들을 strategy.scope 내에 생성

In [ ]:
with strategy.scope():

    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE) 
    train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
  
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE) 
    test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

# 모델 만들기
> tf.keras.Sequential을 사용해서 모델을 생성

In [ ]:
def create_model():
  model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
  ])
  
  return model

In [ ]:
# 체크 포인트를 저장하기 위해 체크포인트 디렉토리 생성
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')

# 손실 함수 정의하기
> 일반적으로, GPU/CPU 비율이 1인 단일 장치에서 손실은 입력 배치(batch)의 샘플 개수로 나누어 진다.
tf.distribute.Strategy를 사용할 때, 손실은 어떻게 계산되어야 할까?
* 예를들면, 4개의 GPU가 있고 입력 배치 크기가 64라고 하죠. 입력 배치 하나가 여러 개의 장치(4개의 GPU)에 분배됩니다. 각 장치는 배치 크기가 16인 입력을 받는다.
* 각 장치에 있는 모델은 해당 입력에 대해 정방향 계산(forward pass)을 수행하고 손실을 계산합니다. 손실을 장치에 할당된 입력 샘플의 수(BATCH_SIZE_PER_REPLICA = 16)로 나누는 것이 아니라 GLOBAL_BATCH_SIZE(64)로 나누어야 한다.
* 위와 같이 계산하는 이유는 그래디언트들이 각 장치에서 계산된 다음, **`모든 장치를 동기화하기 위해 이 그래디언트 값들을 전부 더하기 때문`**이다.

In [ ]:
with strategy.scope():
    # reduction을 `none`으로 설정합니다. 그래서 우리는 축소를 나중에 하고,
    # GLOBAL_BATCH_SIZE로 나눌 수 있습니다.
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      reduction = tf.keras.losses.Reduction.NONE)

